In [115]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlrd

In [155]:
def convert_sheetdata(fname,sh):
    d = pd.read_excel(fname,sheet_name=sh)
    got_it = 0
    ends = []
    for i in d.index:
        if d.loc[i,'Unnamed: 8'] == 'TOTAL':
            got_it += 1
            ends.append(i)
            if got_it == 2:
                break
    starts = [3,ends[0]+4]
    d_new = pd.concat([d.loc[starts[0]:ends[0]-1,:],d.loc[starts[1]:ends[1]-1,:]])
    d_new.reset_index(inplace=True)
    d_new.drop('index',inplace=True,axis=1)
    odd = d_new.loc[[i for i in d_new.index if i%2==1],['Unnamed: '+str(i) for i in range(5,9)]+['Unnamed: 10']]
    odd.reset_index(inplace=True)
    odd.drop('index',inplace=True,axis=1)
    odd.columns = [i+'_unloading' for i in odd.columns]
    d_new.drop([i for i in d_new.index if i%2==1],inplace=True,axis=0)
    d_new.reset_index(inplace=True)
    d_new.drop('index',inplace=True,axis=1)
    new_data = pd.concat([d_new,odd],axis=1)
    new_data.drop(['Unnamed: 4','Unnamed: 10'],axis=1,inplace=True)
    cols = ['Line','Client','Placement Time','Destination','JNPT loading','GTI loading','NSICT loading','BMCT loading','Total loading','Completion Time','Removal Time','Loading dtn','Removal dtn','JNPT unloading','GTI unloading','NSICT unloading','BMCT unloading','Total unloading']
    new_data.columns = cols
    new_data = new_data[['Line','Client','Destination','Placement Time','Completion Time','Removal Time','Loading dtn','Removal dtn','JNPT loading','GTI loading','NSICT loading','BMCT loading','Total loading','JNPT unloading','GTI unloading','NSICT unloading','BMCT unloading','Total unloading']]
    new_data = new_data[new_data['Loading dtn'].notna()]
    new_data = new_data[new_data['Removal dtn'].notna()]
    new_data.reset_index(inplace=True)
    new_data.drop('index',inplace=True,axis=1)
    new_data['Loading dtn'] = new_data['Completion Time']-new_data['Placement Time']
    new_data['Removal dtn'] = new_data['Removal Time']-new_data['Completion Time']
    new_data['Loading dtn'] = new_data['Loading dtn'].apply(lambda x: x.seconds/3600)
    new_data['Removal dtn'] = new_data['Removal dtn'].apply(lambda x: x.seconds/3600)
    return new_data

In [159]:
def iterate_sheetnames(fname,month):
    xls = xlrd.open_workbook(fname, on_demand=True)
    sheet_names = xls.sheet_names()
    done = 0
    for i in sheet_names:
        if i[3:6] == month:
            print("  Adding data for sheet ",i)
            d = convert_sheetdata(fname,i)
            if done == 0:
                ans = d
                done = 1
            else:
                ans = pd.concat([ans,d],axis=0)
    return ans

In [173]:
# 'data/JAN=2022.xlsx'
months = ['JAN','FEB','MAR']
done = 0
for m in months:
    print("Month :",m)
    path = 'data/'+m+'=2022.xlsx'
    d = iterate_sheetnames(path,m)
    if done == 0:
        ans = d
        done += 1
    else:
        ans = pd.concat([ans,d],axis=0)
        ans.reset_index(inplace=True)
        ans.drop('index',axis=1)
ans.drop(['level_0','index'],axis=1,inplace=True)
print(len(ans))
ans.to_csv('data/Individual_train_data.csv')
ans.head(10)

    

Month : JAN
  Adding data for sheet  01=JAN
  Adding data for sheet  02=JAN
  Adding data for sheet  03=JAN
  Adding data for sheet  04=JAN
  Adding data for sheet  05=JAN
  Adding data for sheet  06=JAN
  Adding data for sheet  07=JAN
  Adding data for sheet  08=JAN
  Adding data for sheet  09=JAN
  Adding data for sheet  10=JAN
  Adding data for sheet  11=JAN 
  Adding data for sheet  12=JAN
  Adding data for sheet  13=JAN 
  Adding data for sheet  14=JAN
  Adding data for sheet  15=JAN
  Adding data for sheet  16=JAN
  Adding data for sheet  17=JAN 
  Adding data for sheet  18=JAN
  Adding data for sheet  19=JAN
  Adding data for sheet  20=JAN
  Adding data for sheet  21=JAN
  Adding data for sheet  22=JAN
  Adding data for sheet  23=JAN
  Adding data for sheet  24=JAN
  Adding data for sheet  25=JAN 
  Adding data for sheet  26=JAN
  Adding data for sheet  27=JAN
  Adding data for sheet  28=JAN 
  Adding data for sheet  29=JAN
  Adding data for sheet  30=JAN
  Adding data for sheet

,Line,Client,Destination,Placement Time,Completion Time,Removal Time,Loading dtn,Removal dtn,JNPT loading,GTI loading,NSICT loading,BMCT loading,Total loading,JNPT unloading,GTI unloading,NSICT unloading,BMCT unloading,Total unloading
0,1,CONR,DRTA,2021-12-31 06:20:00,2022-01-01 08:15:00,2022-01-01 09:10:00,1.916667,0.916667,4,32,14,40,90,8,13,23,36,80
1,8,CONR,SNF,2021-12-31 08:05:00,2022-01-01 08:30:00,2022-01-01 09:05:00,0.416667,0.583333,7,0,53,30,90,0,7,68,6,81
2,B4,CONR,MB,2021-12-31 23:30:00,2022-01-01 07:30:00,2022-01-01 08:05:00,8.000000,0.583333,0,0,0,90,90,0,36,43,11,90
3,G1,CONR,BNGD,2022-01-01 02:40:00,2022-01-01 10:45:00,2022-01-01 12:30:00,8.083333,1.750000,6,60,24,0,90,3,28,43,16,90
4,B1,CONR,DLB,2022-01-01 05:50:00,2022-01-01 19:15:00,2022-01-01 20:20:00,13.416667,1.083333,2,10,0,78,90,16,18,25,31,90
5,G3,CONR,TKD,2022-01-01 07:30:00,2022-01-01 14:30:00,2022-01-01 15:30:00,7.000000,1.000000,0,90,0,0,90,0,77,6,7,90
6,B4,CONR,BRC,2022-01-01 11:15:00,2022-01-01 23:45:00,2022-01-02 05:15:00,12.500000,5.500000,0,0,0,90,90,0,11,23,56,90
7,G1,CONR,DRTA,2022-01-01 13:50:00,2022-01-01 23:50:00,2022-01-02 03:20:00,10.000000,3.500000,0,20,36,4,60,0,0,12,78,90
8,G3,CONR,DER,2022-01-01 16:30:00,2022-01-01 23:55:00,2022-01-02 02:15:00,7.416667,2.333333,7,53,20,0,80,0,34,30,16,80
9,5,NCP 01,MRWN,2021-12-31 18:20:00,2022-01-01 02:20:00,2022-01-01 03:05:00,8.000000,0.750000,0,0,77,13,90,2,9,51,28,90
